<a href="https://colab.research.google.com/github/vinurajd/google_collab_DL/blob/master/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
tfds.disable_progress_bar()

import math
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import logging

logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
data_set, meta_data = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = data_set['train'], data_set['test']

In [7]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [8]:
num_train_eg = meta_data.splits['train'].num_examples
num_test_eg = meta_data.splits['test'].num_examples
print(" Training egs: ",str(num_train_eg)," test egs:",str(num_test_eg))

 Training egs:  60000  test egs: 10000


In [9]:
def normalize_ip_img(img,labels):
  img = tf.cast(img,tf.float32)
  img = img/255
  return img, labels

In [10]:
train_dataset=train_dataset.map(normalize_ip_img)
test_dataset=test_dataset.map(normalize_ip_img)

In [ ]:
plt.figure(figsize=(10,10))
i=0
for (img, lab) in test_dataset.take(25):
  img = img.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(img,cmap=plt.cm.binary)
  plt.xlabel(class_names[lab])
  i=i+1
plt.show()


In [12]:
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 201,866
Trainable params: 201,866
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=['accuracy'])

In [15]:
BATCH_SIZE=32
train_dataset=train_dataset.cache().repeat().shuffle(num_train_eg).batch(BATCH_SIZE)
test_dataset=test_dataset.cache().batch(BATCH_SIZE)

In [16]:
model.fit(train_dataset,epochs=5,steps_per_epoch=math.ceil(num_train_eg/BATCH_SIZE))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4750 - accuracy: 0.8272
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3667 - accuracy: 0.8645
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3213 - accuracy: 0.8817
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3064 - accuracy: 0.8845
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2860 - accuracy: 0.8928


In [17]:
test_loss, test_accuracy = model.evaluate(test_dataset,steps=math.ceil(num_test_eg/BATCH_SIZE))
print("Test Accuracy is :", test_accuracy)

313/313 [==============================] - 2s 6ms/step - loss: 0.3644 - accuracy: 0.8698
Test Accuracy is : 0.8697999715805054


In [18]:
for test_images, test_labels in test_dataset.take(1):
  test_images=test_images.numpy()
  test_labels = test_labels.numpy()
  pred = model.predict(test_images)

In [ ]:
pred.shape

In [ ]:
pred[0]